In [1]:
!pip install rise

In [2]:
from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path
path = Path.home() / ".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
cm.update(
    "rise",
    {
        "theme": "sky",
        "transition": "zoom",
        "start_slideshow_at": "selected",
         "rise": {"width": "90%", "height": "90%"}
     }
)

{'theme': 'sky',
 'transition': 'zoom',
 'start_slideshow_at': 'selected',
 'rise': {'width': '90%', 'height': '90%'}}

# The Tata Econometrics Group

## The Gravity Model of Trade

## Motivation

- Our Indian Link & Economic History 

- Research Question: "What are the key insights we can obtain when comparing colonised and not colonised countries on their bilateral trade today?" 
    
    - Hypothesis: 	* *The duration of a colony determines the volume of bilateral trade such that a longer a nation       colonised, the less the bilateral trade. 
    
- (Working) Papers:
    - Mitchener & Weidenmier (2008)
        - Trade Flow factors, from railroads 
    - Nguyen & Wu (2019)
        - Preferential Trade Agreements 

- Our Regression equation:

### ln(Exportsij) = β0 + β1lnYi+ β2lnYj + β3lnDij + β4ELTi + β5ESTi + β4BLTi+ β5BSTi + β6ln(TOij) + β7comcol  + εij

## Approach

- Split into two pairs by timezones

- Our "Tools":
    - Linjun as our Guru
        - Whiteboard 
        - Theory
        - Technical (GroupBy Functions)
    
    - Microsoft Teams/Zoom
   
    - Google Colab
    
    - Github for the Final steps
    


In [3]:
#Tushar Mehra

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

gravity = pd.read_excel('https://www.dropbox.com/s/2uha8rwc8bngcsz/servicesdataset%202.xlsx?dl=1')

gravity.head()

gravity.shape

gravity.describe()

gravity['trade_restrictiveness'] = gravity['ent_cost_imp'] + gravity['ent_cost_exp']
#Adding cost oftrade to importer and cost of trade to exporter together to measure trade restrictiveness (0 means none, more than 2 means virtually impossible to trade) 

gravity = gravity.drop(['contig','comlang_off','comlang_ethno','colony','distcap','etcr_exp','etcr_imp','lat_exp','lat_imp'], axis=1)

colony= pd.read_excel('https://www.dropbox.com/s/216s24p3f8aj8ni/Copy%20of%20Colonial_transformation_data%20%281%29.xls?dl=1')

colony.head()

colony.describe()

colony.shape

gravity.columns

colony.columns

def clean_col(col):
  col = col.strip()
  col = col.replace('Country Name', 'Country Name - Importer')
  col = col.replace('Main colonial "motherland", source: Ziltener/KŸnzler', 'Colonial Motherland - Importer')
  col = col.replace('onset of colonialism, source: Ziltener/KŸnzler 2008', 'Year of Colonisation - Importer')
  col = col.replace('end of colonialism, source: Ziltener/KŸnzler 2008', 'Year of Independence - Importer')
  col = col.replace('COLYEARS', 'Years under Colonisation - Importer')
  return col

new_columns = []
for c in colony.columns:
    clean_c = clean_col(c)
    new_columns.append(clean_c)

colony.columns = new_columns

colony_series_dropper = colony.columns.to_series()['Violent Colonization, Wars of Defence etc; Source: own coding; cf. COW Dataset (':'colonial borders split ethnic groups, cut traditional space of close interaction']
colony = colony.drop(colony_series_dropper, axis=1)
#Dropping extra data points that are not going to be needed for our regression; had to convert columns to a series to be able to drop a range of columns 


combined_imp = gravity.merge(colony, left_on = 'imp', right_on = 'Country Code World Bank', how = 'left')
combined_imp.tail()

def clean_col_2(col):
  col = col.strip()
  col = col.replace('Country Name - Importer', 'Country Name - Exporter')
  col = col.replace('Colonial Motherland - Importer', 'Colonial Motherland - Exporter')
  col = col.replace('Year of Colonisation - Importer', 'Year of Colonisation - Exporter' )
  col = col.replace('Year of Independence - Importer', 'Year of Independence - Exporter')
  col = col.replace('Years under Colonisation - Importer', 'Years under Colonisation - Exporter')
  return col

new_columns = []
for c in colony.columns:
    clean_c = clean_col_2(c)
    new_columns.append(clean_c)

colony.columns = new_columns

# note here that to merge two things, you need to provide a reference column so the computer knows what you're merging based on, i.e. to associate A in dataframe 1 with A in dataframe 2.
# hence ask yourself if you're merging based on exp or imp in the gravity dataframe, and which associatetd column in the colonisation dataframe.

# after that, read the .merge reference here https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.merge.html or I can provide more hints, if you would like.


combined = combined_imp.merge(colony, left_on = 'exp', right_on = 'Country Code World Bank', how = 'left')
combined.tail()

combined.head()

imp_short_term_colony_bool = combined.loc[:, 'Years under Colonisation - Importer'] <= 100
imp_long_term_colony_bool = combined['Years under Colonisation - Importer'] > 100
exp_short_term_colony_bool = combined['Years under Colonisation - Exporter'] <= 100
exp_long_term_colony_bool = combined['Years under Colonisation - Exporter'] > 100
#Time period of 100 years determined by seeing the mean and median of the data from colony.describe() above

combined.loc[imp_short_term_colony_bool,:]

combined.loc[imp_long_term_colony_bool, :]

combined.shape

GBR_test_bool = (combined['imp'] == 'GBR')
#Have this boolean for testing purposes to see what happens to non-colonised countries in our data set


combined.loc[GBR_test_bool, :]
#This is the test to see what happens when you examine the importing data for GBR, a country that was never colonised


combined['imp_short_term_colony'] = 0
combined['imp_long_term_colony'] = 0
combined['exp_short_term_colony'] = 0
combined['exp_long_term_colony'] = 0
#Do all the coding for short-term and long-term colonies in the same cells so that they are easier to keep track of

combined.loc[imp_short_term_colony_bool,'imp_short_term_colony'] = 1
combined.loc[imp_long_term_colony_bool,'imp_long_term_colony'] = 1
combined.loc[exp_short_term_colony_bool,'exp_short_term_colony'] = 1
combined.loc[exp_long_term_colony_bool,'exp_long_term_colony'] = 1

colony_duration_test_bool = (combined['imp'] == 'IND')
combined.loc[colony_duration_test_bool, :]
#This boolean is to test whether the dummy of short-term and long-term colony has worked or not; can change the three-letter imp code and see the short-term colony column

combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'P','Colonial Motherland - Importer'] = 'PRT'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'P','Colonial Motherland - Exporter'] = 'PRT'
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'UK', 'Colonial Motherland - Importer'] = 'GBR'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'UK', 'Colonial Motherland - Exporter'] = 'GBR'
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'B', 'Colonial Motherland -Importer'] = 'BEL'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'B', 'Colonial Motherland -Exporter'] = 'BEL'  
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'F', 'Colonial Motherland - Importer'] = 'FRA'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'F', 'Colonial Motherland - Exporter'] = 'FRA'
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'I', 'Colonial Motherland - Importer'] = 'ITA'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'I', 'Colonial Motherland - Exporter'] = 'ITA'
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'SP', 'Colonial Motherland - Importer'] = 'ESP'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'SP', 'Colonial Motherland -Exporter'] = 'ESP'
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'NL', 'Colonial Motherland - Importer'] = 'NLD'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'NL', 'Colonial Motherland - Exporter'] = 'NLD' 
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'JP', 'Colonial Motherland - Importer'] = 'JPN'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'JP', 'Colonial Motherland - Exporter'] = 'JPN'
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'US', 'Colonial Motherland - Importer'] = 'USA'    
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'US', 'Colonial Motherland - Exporter'] = 'USA'
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'AU', 'Colonial Motherland - Importer'] = 'AUS'
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'AU', 'Colonial Motherland - Exporter'] = 'AUS'
combined.loc[combined.loc[:,'Colonial Motherland - Importer'] == 'UK/F','Colonial Motherland - Importer'] = 'GBR/FRA' 
combined.loc[combined.loc[:,'Colonial Motherland - Exporter'] == 'UK/F','Colonial Motherland - Exporter'] = 'GBR/FRA' 
combined.loc[combined.loc[:,'Colonial Motherland -Importer'] == 'x','Colonial Motherland - Importer'] = 'INDIRECT'
combined.loc[combined.loc[:,'Colonial Motherland -Exporter'] == 'x','Colonial Motherland - Exporter'] = 'INDIRECT'

combined['Colonial Motherland - Importer'].unique()

combined['Colonial Motherland - Exporter'].unique()

combined.shape

def combine_partner_name(x):
  partner_1 = x['imp']
  partner_2 = x['exp']

  partner_list = [partner_1, partner_2]
  partner_list.sort()

  x['partner-list'] = partner_list[0] + "-" + partner_list[1]
  return x

combined = combined.apply(combine_partner_name, axis = 1)

combined.tail()

combined.shape

trade_data = combined.groupby(['partner-list'])['trade'].sum().reset_index()
trade_data.head()
#If you don't use the reset index method, then you would just have a series where the partner-list is the index. The reset index takes the partner list out and makes it a separate column

combined_specific = combined[['partner-list', 'comcol', 'curcol', 'col45', 'dist','distw', 'distwces', 'gdp_exp', 'gdp_imp', 'ent_cost_imp', 'ent_cost_exp','trade_restrictiveness', 'imp_short_term_colony',	'imp_long_term_colony','exp_short_term_colony','exp_long_term_colony']].drop_duplicates(subset = ['partner-list'])
combined_specific.rename(columns={'gdp_exp': 'gdp_1', 'gdp_imp':'gdp_2','imp_short_term_colony':'short_term_colony_1', 'exp_short_term_colony': 'short_term_colony_2', 'imp_long_term_colony':'long_term_colony_1', 'exp_long_term_colony': 'long_term_colony_2' })
#Dropping duplicates for colony data in the subset of partner list doesn't matter since the only case where there would be a duplicate to be dropped is if neither are colonies, both are short-term or both are long-term

combined_trade_data = pd.merge(trade_data, combined_specific, on ='partner-list', how = 'inner')
combined_trade_data[combined_trade_data['partner-list'] == 'GBR-IND']

combined_trade_data = combined_trade_data.rename(columns={'gdp_exp': 'gdp_1', 'gdp_imp':'gdp_2','imp_short_term_colony':'short_term_colony_1', 'exp_short_term_colony': 'short_term_colony_2', 'imp_long_term_colony':'long_term_colony_1', 'exp_long_term_colony': 'long_term_colony_2', 'ent_cost_imp':'ent_cost_1','ent_cost_exp':'ent_cost_2' })
combined_trade_data.head()

combined_trade_data.shape

combined_trade_data['either_short_term_colony'] = combined_trade_data['short_term_colony_1'] + combined_trade_data['short_term_colony_2']
combined_trade_data['either_long_term_colony'] = combined_trade_data['long_term_colony_1'] + combined_trade_data['long_term_colony_2']
combined_trade_data.value_counts('either_short_term_colony')


combined_trade_data.value_counts('either_long_term_colony')

combined_trade_data.loc[combined_trade_data['either_short_term_colony'] == 2, 'both_short_term_colony'] = 1
combined_trade_data.loc[combined_trade_data['either_short_term_colony'] == 2]

combined_trade_data.loc[combined_trade_data['either_long_term_colony'] == 2, 'both_long_term_colony'] = 1
combined_trade_data.loc[combined_trade_data['either_long_term_colony'] == 2]

combined_trade_data.shape

combined_trade_data['either_short_term_colony'] = combined_trade_data['either_short_term_colony'].replace(2,0)
combined_trade_data['either_long_term_colony'] = combined_trade_data['either_long_term_colony'].replace(2,0)

combined_trade_data.value_counts('either_short_term_colony')

combined_trade_data.value_counts('either_long_term_colony')

combined_trade_data['both_short_term_colony'] = combined_trade_data['both_short_term_colony'].replace(np.nan,0)
combined_trade_data['both_long_term_colony'] = combined_trade_data['both_long_term_colony'].replace(np.nan,0)

combined_trade_data.value_counts('both_long_term_colony')

combined_trade_data.value_counts('both_short_term_colony')

combined_trade_data.shape

combined_trade_data.tail()

combined_trade_data['ln_GDP_1'] = np.log(combined_trade_data['gdp_1'])
combined_trade_data['ln_GDP_2'] = np.log(combined_trade_data['gdp_2'])
combined_trade_data['ln_distance'] = np.log(combined_trade_data['dist'])
combined_trade_data = combined_trade_data[combined_trade_data['trade'] != 0]
combined_trade_data['ln_trade'] = np.log(combined_trade_data['trade'])
combined_trade_data['ln_trade_restrictiveness'] = np.log(combined_trade_data['trade_restrictiveness'])

combined_trade_data.to_csv('Clean_Data.csv', index = False)

X = combined_trade_data[['ln_GDP_1', 'ln_GDP_2', 'ln_distance', 'ln_trade_restrictiveness','comcol', 'either_short_term_colony', 'either_long_term_colony', 'both_short_term_colony','both_long_term_colony']]
Y = combined_trade_data['ln_trade']
X = sm.add_constant(X)

regression_1 = sm.OLS(endog=Y, exog=X, missing ='drop')
regression_1_results = regression_1.fit(cov_type = 'HC1')
regression_1_results.summary()



KeyboardInterrupt: 

## Findings
- Being a long-term colony is insignificant to biilateral trade
- Short-term colonies seemingly ally together
- Trade restrictiveness is less of a trade cost than distance
- The traditional gravity model takes precedence over our assumptions/hypothesis

In [ ]:
from IPython.display import Image
Image(filename = 'GDP-Trade.png', width = 400, height = 400)
Image(filename = 'Distance-Trade.png', width = 400, height = 400)


In [ ]:
#Visualisations

## Text for Yash & Aryan 

# Closing Points for the Team